# Tutorial for using the Fitness Landscape Exploration Sandbox

* [Building oracles for landscapes](#section-one)
* [Building a model of the landscape](#section-two)
* [Exploring a landscape with a model](#section-three)
* [Evaluating an exploration algorithm](#section-four)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import editdistance
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
import flexs
from flexs import baselines
import flexs.utils.sequence_utils as s_utils

AttributeError: module 'flexs.baselines' has no attribute 'models'

## 1. Building oracles for landscapes <a class="anchor" id="section-one"></a>

Most FLEXS landscapes have a problem registry: a dictionary of problems, parameters, and metadata that can be used to quickly generate landscapes and set up interesting benchmarks for models and explorers.

Here we select a landscape that will maximize the negative binding energy of sequences of length 14 against a predefined RNA target.

In [ ]:
problem = flexs.landscapes.rna.registry()['L14_RNA1']
problem

For RNA binding landscapes, the registry contains several starting points of diverse quality and sequence identity on which to evaluate exploration algorithms.

In [10]:
wt = problem['starts'][0]
wt

'AUGGGCCGGACCCC'

Now we instantiate the landscape and set the sequence alphabet.

In [11]:
landscape = flexs.landscapes.RNABinding(**problem['params'])
alphabet = s_utils.RNAA

You can now query the landscape for ground truth sequence fitnesses

In [12]:
landscape.get_fitness([wt])

array([0.54374688])

While we show this process on RNA landscapes there are other empirical landscapes that we have similar wrappers around. You can also make your own *custom* landscapes (see examples in `flexs/landscapes`).

Here we demonstrate a rosetta folding landscape, the BERT GFP landscape, and an additive AAV packaging landscape.

In [14]:
rosetta_problem = flexs.landscapes.rosetta.registry()['3msi']
rosetta_problem

{'params': {'pdb_file': '/Users/stewartslocum/Downloads/FLSD-Sandbox/flexs/landscapes/data/rosetta/3msi.pdb',
  'sigmoid_center': -3,
  'sigmoid_norm_value': 12},
 'starts': {'ed_3_wt': 'MAQASVVANQLIPINTHLTLVMMRSEVVTYVHIPAEDIPRLVSMDVNRAVPLGTTLMPDMVKGYAA',
  'ed_5_wt': 'MAQASVVFNQLIPINTHLTLVMMRFEVVTPVGCPAMDIPRLVSQQVNRAVPLGTTLMPDMVKGYAA',
  'ed_7_wt': 'WAQRSVVANQLIPINTGLTLVMMRSELVTGVGAPAEDIPRLVSMQVNRAVPLGTTNMPDMVKGYAA',
  'ed_12_wt': 'RAQESVVANQLIPILTHLTQKMSRRFVVTPVGIPAEDIPRLVNAQVDRAVPLGTTLMPDMDKGYAA',
  'ed_27_wt': 'MRRYSVIAYQERPINLHSTLTFNRSEVPWPVNRPASDAPRLVSMQNNRSVPLGTKLPEDPVCRYAL'}}

In [16]:
protein_landscape = flexs.landscapes.RosettaFolding(**rosetta_problem['params'])

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.mac.python37.Release 2020.02+release.22ef835b4a2647af94fcd6421a85720f07eddf12 2020-01-05T17:31:56] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [17]:
protein_wt = protein_landscape.wt_pose.sequence()
protein_wt

'MAQASVVANQLIPINTHLTLVMMRSEVVTPVGIPAEDIPRLVSMQVNRAVPLGTTLMPDMVKGYAA'

In [18]:
protein_landscape.get_fitness([protein_wt])

array([0.3494302], dtype=float32)

In [19]:
bert_gfp_landscape = flexs.landscapes.BertGFPBrightness()

In [20]:
gfp_sequence = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVT" \
               "TLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIE" \
               "LKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNT" \
               "PIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

bert_gfp_landscape.get_fitness([gfp_sequence])

array([3.57247734])

In [21]:
tf_binding_problem = flexs.landscapes.tf_binding.registry()['VAX2_REF_R2']
tf_binding_problem

{'params': {'landscape_file': '/Users/stewartslocum/Downloads/FLSD-Sandbox/flexs/landscapes/data/tf_binding/VAX2_REF_R2_8mers.txt'},
 'starts': ['GCTCGAGC',
  'GCGCGCGC',
  'TGCGCGCC',
  'ATATAGCC',
  'GTTTGGTA',
  'ATTATGTT',
  'CAGTTTTT',
  'AAAAATTT',
  'AAAAACGC',
  'GTTGTTTT',
  'TGCTTTTT',
  'AAAGATAG',
  'CCTTCTTT',
  'AAAGAGAG']}

In [22]:
tf_binding_landscape = flexs.landscapes.TFBinding(**tf_binding_problem['params'])

In [23]:
tf_binding_landscape.get_fitness(["CGTATCAA"])

array([0.45653133])

In [24]:
additive_aav_problem = flexs.landscapes.additive_aav_packaging.registry()['liver']
additive_aav_problem

{'params': {'phenotype': 'liver', 'start': 450, 'end': 540}}

In [25]:
additive_aav_landscape = flexs.landscapes.AdditiveAAVPackaging(**additive_aav_problem['params'])

In [26]:
additive_aav_landscape.get_fitness([additive_aav_landscape.wild_type])

array([0.5])

## 2. Building a model of the landscape <a class="anchor" id="section-two"></a>

In reality, you don't have access to the ground truth landscape oracles except through costly experiments. Instead you can access an approximate version of it either by modeling the landscape through your current observations. In section 2.1 we use what we call a "noisy abstract model" to simulate the situation where you have a model with arbitrary noise. In section 2.2 we show some standard neural nets and use them as models of the landscape. It is easy to implement your own model. All they need to do is to inherit from `flexs.Model` and implement `_fitness_function(sequences)` which gives you the model's estimate of a sequences fitness, and `train(sequences, labels)`.

### 2.1. Null model

The null model is the most simple model of the landscape. At each epoch, the model guesses the fitness of a variant by drawing a random sample from the exponential distribution $\lambda e^{-\lambda x}$, with $\lambda = \mathbb{E}[f_\text{measured_sequences}]$. 

In [27]:
null_model = baselines.models.NullModel()

We want to "measure" the true fitness of at least one sequence to start with (otherwise we can't have a model). We can do that by calling `measure_true_landscape` on the noisy model. This will charge you a "measurment cost", but in return tells you the true fitness of the sequence. When starting out or reusing the landscape, `reset` lets you zero the cost in the landscape, and if you pass it any sequences, it will measure those.  

let's ask it for the fitness of a random sequence (we will show more complex results later).

In [28]:
null_model.get_fitness([wt]), landscape.get_fitness([wt])

(array([0.00398003]), array([0.54374688]))

### 2.2 Abstract model 

This is an abstract model that "emulates" a real model of the landscape. It works by guessing the fitness of a new sequence as a combination of it's real fitness and fitness of the closest known neighbor. 

To build a noisy model, you need to have an oracle, and a `signal_strength` $\alpha \in [0,1]$. This parameter $\alpha$ determines how much your oracle signal is corrupted by noise:

$f_{model}= \alpha^d f_{oracle} + (1-\alpha^d)\epsilon$

here $d$ is the distance to the closest measured neighbor, and $\epsilon$ is a sample from an exponential distribution $\lambda e^{-\lambda x}$, with $\lambda = f_\text{closest_neighbor}$ (for each queried sample, this is sampled once and then fixed for that batch). 

In [29]:
noisy_models = [baselines.models.NoisyAbstractModel(landscape, signal_strength=ss) for ss in [1, 0.9, 0.75, 0]]
for model in noisy_models:
    model.train([wt], landscape.get_fitness([wt]))

Let's measure the fitness of some samples through the model

In [66]:
val_x = np.array(list(set(s_utils.generate_random_mutant(wt, 0.2*(i%5), alphabet=alphabet)
                  for i in range(1000))))
val_y = landscape.get_fitness(val_x)

list(zip(val_x[:5], noisy_models[2].get_fitness(val_x[:5])))

[('AUCUGACGGUCCCC', 0.23380105789573236),
 ('AGGGGCCGAACCCU', 0.30591261245766854),
 ('AUGGACCGGACUCC', 0.4692614267816372),
 ('AUCGCCAUGCGCUU', 0.3345053122132554),
 ('AUGGCCCUAACGUG', 0.1930244408945141)]

In [67]:
distances = [editdistance.eval(seq, wt) for seq in val_x]

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
ax[0].set_title('Edit distance distribution from wild-type (validation set)')
sns.distplot(distances, kde=False, ax=ax[0])
ax[1].set_title('Fitness distribution (validation set)')
sns.distplot(val_y, kde=True, ax=ax[1])
plt.show()

Now we will examine the extrapolation ability of noisy abstract models at different signals strengths.

In [68]:
import scipy.stats

def get_corr_per_distance(seqs, labels, preds):    
    corrs = []
    edit_distances = np.array([editdistance.eval(seq, wt) for seq in seqs])
    
    for dist in range(1, 11):
        seqs_at_dist = edit_distances == dist
        
        if seqs_at_dist.sum() > 50:
            corrs.append(
                scipy.stats.pearsonr(labels[seqs_at_dist], preds[seqs_at_dist])[0]**2
            )
        else:
            corrs.append(np.nan)
            
    return corrs

In [69]:
corrs_noisy_neighbors = [get_corr_per_distance(val_x, val_y, noisy_model.get_fitness(val_x))
                         for noisy_model in noisy_models]
corrs_null = get_corr_per_distance(val_x, val_y, null_model.get_fitness(val_x))

In [70]:
plt.figure(figsize=(10,5), dpi=300)
plt.title('Noisy abstract model extrapolation ability')
plt.plot(range(1,11), corrs_noisy_neighbors[0], 'o-', label="$\phi'_{1}$")
plt.plot(range(1,11), corrs_noisy_neighbors[1], 'o-', label="$\phi'_{0.9}$")
plt.plot(range(1,11), corrs_noisy_neighbors[2], 'o-', label="$\phi'_{0.5}$")
plt.plot(range(1,11), corrs_noisy_neighbors[3], 'o-', label="$\phi'_{0.0}$")
plt.plot(range(1,11), corrs_null, 'o-', label="null")
plt.ylabel("$r^2$ between model and ground truth")
plt.xlabel("Distance from wt")
plt.legend()
plt.show()

### 2.3 Models of landscapes built from data

Similarly, we can approximate the landscape by training a model on it. Some basic architectures are provided for convenience.

For real models, it's more sensible to train them on more than just the WT

In [71]:
train_x = np.array(list(set(s_utils.generate_random_mutant(wt, 0.2*(i%5), alphabet)
                                 for i in range(0, 20000))))
train_y = landscape.get_fitness(train_x)
print (f'training data consisted of {len(train_x)} unique sequences')

training data consisted of 13250 unique sequences


In [72]:
linear_regression = baselines.models.LinearRegression(alphabet=alphabet)
linear_regression.train(train_x, train_y)

linear_regression_preds = linear_regression.get_fitness(val_x)
corrs_linear = get_corr_per_distance(val_x, val_y, linear_regression_preds)

print(f"Overall correlation: {scipy.stats.pearsonr(linear_regression_preds, val_y)[0]**2}")

Overall correlation: 0.7883118558928867


In [73]:
random_forest = baselines.models.RandomForest(alphabet=alphabet)
random_forest.train(train_x, train_y)

random_forest_preds = random_forest.get_fitness(val_x)
corrs_rf = get_corr_per_distance(val_x, val_y, random_forest_preds)

print(f"Overall correlation: {scipy.stats.pearsonr(random_forest_preds, val_y)[0]**2}")

Overall correlation: 0.8300053912924367


In [74]:
mlp = baselines.models.MLP(len(wt), hidden_size=100, alphabet=alphabet, loss='MSE')
mlp.train(train_x, train_y)

mlp_preds = mlp.get_fitness(val_x)
corrs_mlp = get_corr_per_distance(val_x, val_y, mlp_preds)

print(f"Overall correlation: {scipy.stats.pearsonr(mlp_preds, val_y)[0]**2}")

Overall correlation: 0.8477103380352756


In [75]:
cnn = baselines.models.CNN(len(wt), num_filters=32, hidden_size=100, alphabet=alphabet, loss='MSE')
cnn.train(train_x, train_y)

cnn_preds = cnn.get_fitness(val_x)
corrs_cnn = get_corr_per_distance(val_x, val_y, cnn_preds)

print(f"Overall correlation: {scipy.stats.pearsonr(cnn_preds, val_y)[0]**2}")

Overall correlation: 0.8975465144142935


We can easily ensemble these models of the landscape as well. 

In [79]:
cnn_ensemble = flexs.Ensemble([
    baselines.models.CNN(sequence_length, alphabet=alphabet, num_filters=32, hidden_size=100, loss='MSE')
    for i in range(3)
])
cnn_ensemble.train(train_x, train_y)

cnn_ensemble_preds = cnn_ensemble.get_fitness(val_x)
corrs_cnn_ensemble = get_corr_per_distance(val_x, val_y, cnn_ensemble_preds)

print(f"Overall correlation: {scipy.stats.pearsonr(cnn_ensemble_preds, val_y)[0]**2}")

Overall correlation: 0.9197448060389077


In [77]:
plt.figure(figsize=(10,5), dpi=300)
plt.title('Model extrapolation ability')
plt.plot(range(1,11), corrs_linear, 'o-', label="linear regression")
plt.plot(range(1,11), corrs_rf, 'o-', label="random forest")
plt.plot(range(1,11), corrs_mlp, 'o-', label="MLP")
plt.plot(range(1,11), corrs_cnn, 'o-', label="CNN")
plt.plot(range(1, 11), corrs_cnn_ensemble, 'o-', label="CNN Ensemble (x3)")
plt.plot(range(1,11), corrs_noisy_neighbors[1], 'o-', label="$\phi'_{0.9}$")
plt.ylabel("$r^2$ between between model and ground truth")
plt.xlabel("Distance from WT")
plt.legend()
plt.ylim(0, 1)
plt.show()

In [78]:
plt.figure(figsize=(10,5), dpi=300)

sort_order = np.argsort(val_y[:100])

plt.scatter(range(len(sort_order)), val_y[sort_order], c="k", s=15, label="Ground Truth")
plt.scatter(range(len(sort_order)), linear_regression_preds[sort_order], c="royalblue", s=5, label="Linear Regression")
plt.scatter(range(len(sort_order)), mlp_preds[sort_order], c="cyan", s=5, label="MLP")
plt.scatter(range(len(sort_order)), cnn_preds[sort_order], c="red", s=5, label="CNN")
plt.scatter(range(len(sort_order)), cnn_ensemble_preds[sort_order], c="salmon", s=5, label="CNN Ensemble (x3)")

plt.legend()
plt.show()

Now that we have models, we can use them with an exploration algorithm to search the landscape. 

## 3. Exploring the landscape with a model <a class="anchor" id="section-three"></a>

All exploration algorithms in this package inherit from `Base_explorer` which performs some house-keeping tasks. We recommend doing the same when you "bring your own" explorer. It's very easy to implement an explorer in this framework, all it needs to implement is a `propose_sequences` method, that suggest new sequences for which a "costly" measurment to the oracle is performed.  

In [80]:
cnn = baselines.models.CNN(len(wt), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

random_explorer = baselines.explorers.Random(
    cnn,
    rounds=10,
    mu=1,
    starting_sequence=wt,
    sequences_batch_size=100,
    model_queries_per_batch=1000,
    alphabet=alphabet
)

- `sequences_batch_size` denotes the experimental batch size per round
- `model_queries_per_batch` is the number of computational queries to the model that is allowed per round. 
- `alphabet` is the alphabet that is permissive on the current landscape.

In [81]:
random_sequences, metadata = random_explorer.run(landscape)
random_sequences

round: 0, top: 0.4904482960700989
round: 1, top: 0.5734423995018005
round: 2, top: 0.5923985242843628
round: 3, top: 0.6105319261550903
round: 4, top: 0.6105319261550903
round: 5, top: 0.6105319261550903
round: 6, top: 0.6105319261550903
round: 7, top: 0.6105319261550903
round: 8, top: 0.6105319261550903
round: 9, top: 0.6134910583496094
round: 10, top: 0.6134910583496094


sequence  model_score  \
0   MAQASVVANQLIPINTHLTLVMMRSEVVTPVGIPAEDIPRLVSMQV...          NaN   
0   MAQASVVANQLIPINTHLTLVMMRSEVVTPVGIPAEDIPRLVSMQV...     0.573507   
1   MAQASVVANQLIPINTHLTLVMCRSEVVTPVGIPAEDIPRLVSMQV...     0.564480   
2   MAQASVVANQIIPINTHLTLVMMRSEVVTPVGIPAEDDPRLNSMQV...     0.530958   
3   MAYASVVANQLIPINTHLTLVMMRSEVVTPVGIPAEDIPRLVCMQV...     0.571592   
..                                                ...          ...   
95  MAQASVVANQLIPINTHLPLVMMRSEVVTPVGIPAEDWPRLVSMQV...     0.319346   
96  MAQASVVANQLIPIPTHLTSVPMGSEVVTPVGIPAEDIGRLVSMQV...     0.428091   
97  MAQASVAANQLIPINTHLTLVMMRSEVMTPVGIPAEDIPRNVSMQV...     0.285721   
98  MAQASVVANQTIPINTHLTLVMMRSEVVAPVGIPAEDIPRLVSKQV...     0.401472   
99  MAQASVVSNQLIPINTHLTLVVCRSEVVTPVGIPAEDIPRLVSMQV...     0.415994   

    true_score  round  model_cost  measurement_cost  
0     0.490448      0           0                 1  
0     0.364715      1        1001               101  
1     0.464647      1        1001               101  
2     0.375031      1        1001               101  
3     0.370486      1        1001               101  
..         ...    ...         ...               ...  
95    0.213992     10       10010              1001  
96    0.449938     10       10010              1001  
97    0.303382     10       10010              1001  
98    0.379333     10       10010              1001  
99    0.443197     10       10010              1001  

[1001 rows x 6 columns]

Let's see what was printed:
- The number of **rounds** represents how many experiments was done
- The **cost** shows how many total calls to the oracle was made (experimental cost). Because training the model had an overhead, that is accounted in the total cost. 
- The **evals** shows how many total calls to the model was made (computational cost).
- **Top** simply lists the top fitness (cumulative) observed through the exploration

In [16]:
cnn = baselines.models.CNN(len(wt), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

adalead_explorer = baselines.explorers.Adalead(
    cnn,
    rounds=10,
    starting_sequence=wt,
    sequences_batch_size=100,
    model_queries_per_batch=1000,
    alphabet=alphabet
)

In [17]:
adalead_sequences, metadata = adalead_explorer.run(landscape)
adalead_sequences

round: 0, top: 0.27879076194752395, time: 0s


KeyboardInterrupt: 

In [40]:
cnn = baselines.models.CNN(len(wt), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

genetic_explorer = baselines.explorers.GeneticAlgorithm(
    cnn,
    
    population_size=20,
    parent_selection_strategy='wright-fisher', # roughly corresponds to who gets to mate
    beta=0.01,
    children_proportion=0.2,

    rounds=10,
    starting_sequence=wt,
    sequences_batch_size=100,
    model_queries_per_batch=1000,
    alphabet=alphabet
)

In [41]:
genetic_algo_sequences, metadata = genetic_explorer.run(landscape)
genetic_algo_sequences

round: 0, top: 0.27879076194752395
round: 1, top: 0.5927639336594837
round: 2, top: 0.6733363044747718
round: 3, top: 0.8190401556440864
round: 4, top: 0.940339190418271
round: 5, top: 1.0319481003143287
round: 6, top: 1.0319481003143287
round: 7, top: 1.0319481003143287
round: 8, top: 1.0319481003143287
round: 9, top: 1.0319481003143287
round: 10, top: 1.0319481003143287


sequence  model_score  true_score  round  model_cost  \
0   UCUUGGGGACUUUU          NaN    0.278791      0           0   
0   GCGUGAGGAGAUUU     0.411485    0.327186      1         980   
1   GCGUGAGGAGAUUC     0.400957    0.341720      1         980   
2   GCGUGAGGAGACUU     0.396656    0.360270      1         980   
3   GCGUGAGGAGAUUA     0.396354    0.329829      1         980   
..             ...          ...         ...    ...         ...   
94  GCGCGGGGGCACUG     0.772193    0.781371     10        9800   
95  GCGGGGGGGACCCG     0.772080    0.740955     10        9800   
96  GCGCCGGGGCUCCG     0.771955    0.784065     10        9800   
97  GCGCAGGGGCGCCG     0.771933    0.746344     10        9800   
98  GCACGGGGGGCCCG     0.771828    0.767899     10        9800   

    measurement_cost  
0                  1  
0                100  
1                100  
2                100  
3                100  
..               ...  
94               991  
95               991  
96               991  
97               991  
98               991  

[991 rows x 6 columns]

In [38]:
cnn = baselines.models.CNN(len(wt), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

cmaes_explorer = baselines.explorers.CMAES(
    flexs.LandscapeAsModel(landscape),
    
    population_size=10,
    max_iter=200,
    
    rounds=10,
    starting_sequence=wt,
    sequences_batch_size=100,
    model_queries_per_batch=1000,
    alphabet=alphabet
)

In [39]:
cmaes_sequences, metadata = cmaes_explorer.run(landscape)
cmaes_sequences

round: 0, top: 0.5437468768175177, time: 0.000159s
round: 1, top: 0.6069094266729929, time: 0.495011s
round: 2, top: 0.6069094266729929, time: 0.431063s
round: 3, top: 0.6069094266729929, time: 0.574129s
round: 4, top: 0.6151479950813181, time: 0.665124s
round: 5, top: 0.6783105449367933, time: 0.489589s
round: 6, top: 0.6783105449367933, time: 0.541819s
round: 7, top: 0.6783105449367933, time: 0.558818s
round: 8, top: 0.7057725094705981, time: 0.477559s
round: 9, top: 0.7057725094705981, time: 0.555270s
round: 10, top: 0.7057725094705981, time: 0.596477s


sequence  model_score  true_score  round  model_cost  \
0   AUGGGCCGGACCCC          NaN    0.543747      0           0   
0   AUGGGCCGGACGCC     0.606909    0.606909      1         226   
1   AUGGGCCGGACCCG     0.565716    0.565716      1         226   
2   AUGGGCCGGACCUC     0.560224    0.560224      1         226   
3   AUGGGCCGGACCCC     0.543747    0.543747      1         226   
..             ...          ...         ...    ...         ...   
94  GCGUGUCAUAUAAC     0.260889    0.260889     10        1868   
95  GCGUGUGAAAUAAC     0.260889    0.260889     10        1868   
96  GCGUGUGAAAUGAC     0.260889    0.260889     10        1868   
97  CCGUGUCAAUUAAC     0.258142    0.258142     10        1868   
98  GGGUACCAAAUAUC     0.258142    0.258142     10        1868   

    measurement_cost  
0                  1  
0                100  
1                100  
2                100  
3                100  
..               ...  
94               869  
95               869  
96               869  
97               869  
98               869  

[869 rows x 6 columns]

In [85]:
cnn = baselines.models.CNN(len(wt), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

ppo_explorer = baselines.explorers.PPO(
    flexs.LandscapeAsModel(landscape),
    rounds=10,
    starting_sequence=wt,
    sequences_batch_size=100,
    model_queries_per_batch=1000,
    alphabet=alphabet,
)

In [86]:
ppo_sequences, metadata = ppo_explorer.run(landscape)
ppo_sequences

round: 0, top: 0.4904482960700989, 0:00:00.000185 seconds
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=True)` instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=True)` instead.


round: 1, top: 0.8074898719787598, 0:00:08.214898 seconds
round: 2, top: 0.85374915599823, 0:00:08.390629 seconds
round: 3, top: 0.8910526037216187, 0:00:08.588660 seconds
round: 4, top: 0.9426808953285217, 0:00:08.756873 seconds
round: 5, top: 0.9480541944503784, 0:00:08.827509 seconds
round: 6, top: 0.9760991930961609, 0:00:08.961915 seconds
round: 7, top: 0.9760991930961609, 0:00:08.905312 seconds
round: 8, top: 0.9760991930961609, 0:00:08.798569 seconds
round: 9, top: 0.9768895506858826, 0:00:08.706297 seconds
round: 10, top: 0.9768895506858826, 0:00:08.607299 seconds


sequence  model_score  \
0   MAQASVVANQLIPINTHLTLVMMRSEVVTPVGIPAEDIPRLVSMQV...          NaN   
0   HWQHSFVANQLIKIDTHLTLEQISSEHVTPEGIPAEWAPRFVSRQV...     0.807834   
1   HWQASLVANRLIKIDTPLTSEQLTSEHVSPESIPACWAPRFVSMQV...     0.807490   
2   HWQASLVANRLIKIDTPLTSEQLTSEHVSPESIPAEWAPRFVSMQV...     0.802347   
3   HWQASFVANQLIKIDTHLTLEQITSEHVTPEGIPAEWAPRFVSMQV...     0.787523   
..                                                ...          ...   
94  PDRPDKHWNSTIHGLHNRPQEWNVKDRKKYKCHSGRQDEEWPGPIV...     0.881618   
95  LDTMDKHWNSTYHGLHNTPQERPHKDRKKSKQHSGMTDEIQPGPMC...     0.881491   
96  PDPWDKHWSSTHHGLMNRPGEWPVKDRKVSKCHSYMQDNEWPGPWC...     0.881120   
97  PDRRDKCWNSFHHHLHSRPQEWPVKFRKKSKCHSGMQDTEHPGPWC...     0.880423   
98  PDMPDKHWNSEHHGYHVAPQEWPVKDRTKSKCHSEMQDEEWPGPWC...     0.879773   

    true_score  round  model_cost  measurement_cost  
0     0.490448      0           0                 1  
0     0.786836      1        1002               100  
1     0.731080      1        1002               100  
2     0.807490      1        1002               100  
3     0.790176      1        1002               100  
..         ...    ...         ...               ...  
94    0.859890     10       10020               991  
95    0.872674     10       10020               991  
96    0.871089     10       10020               991  
97    0.880423     10       10020               991  
98    0.890355     10       10020               991  

[991 rows x 6 columns]

In [12]:
dynappo_explorer = baselines.explorers.DynaPPO(
    landscape=landscape,
    env_batch_size=10,
    num_model_rounds=20,
    rounds=10,
    starting_sequence=wt,
    sequences_batch_size=10,
    model_queries_per_batch=10,
    alphabet=alphabet,
)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


In [13]:
dynappo_sequences, metadata = dynappo_explorer.run(landscape)
dynappo_sequences

AttributeError: 'DynaPPOEnsemble' object has no attribute 'name'

In [ ]:
cnn = baselines.models.CNN(len(wt), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

vae = baselines.explorers.VAE(len(wt), alphabet=alphabet, epochs=2, verbose=False)

cbas_explorer = baselines.explorers.CbAS(
    cnn,
    vae,

    rounds=10,
    starting_sequence=wt,
    sequences_batch_size=100,
    model_queries_per_batch=1000,
    alphabet=alphabet,
)

In [40]:
cbas_sequences, metadata = cbas_explorer.run(landscape)
cbas_sequences

round: 0, top: 0.15601505842016555


/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


round: 1, top: 0.5968573029520681
round: 2, top: 0.6266768702280845
round: 3, top: 0.6266768702280845
round: 4, top: 0.6266768702280845
round: 5, top: 0.6266768702280845
round: 6, top: 0.6266768702280845
round: 7, top: 0.6266768702280845
round: 8, top: 0.6465997383450318
round: 9, top: 0.6465997383450318
round: 10, top: 0.6465997383450318


sequence  model_score  true_score  round  model_cost  \
0   GGAUAACAAUUCAU          NaN    0.156015      0           0   
0   GGCUUCCCCUGUCA     0.596857    0.596857      1        1000   
1   CGACCUCCGUUCGC     0.591805    0.591805      1        1000   
2   GGCAGCCCCGUUCA     0.583308    0.583308      1        1000   
3   GGCAACCCCGUUCA     0.569732    0.569732      1        1000   
..             ...          ...         ...    ...         ...   
94  UCCUCCGUCUAUCG     0.394105    0.394105     10       10000   
95  UCUCGUUCUGGGAU     0.393328    0.393328     10       10000   
96  CCAUGCUCUGUGAU     0.393147    0.393147     10       10000   
97  UAAAGCGCUGGGUU     0.392913    0.392913     10       10000   
98  GGCGUCUGGUAUCU     0.392214    0.392214     10       10000   

    measurement_cost  
0                  1  
0                100  
1                100  
2                100  
3                100  
..               ...  
94               991  
95               991  
96               991  
97               991  
98               991  

[991 rows x 6 columns]

In [15]:
cnn = baselines.models.CNN(len(wt), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

dqn_explorer = baselines.explorers.DQN(
    cnn,
    rounds=5,
    starting_sequence=wt,
    sequences_batch_size=100,
    model_queries_per_batch=1000,
    alphabet=alphabet
)

In [16]:
dqn_sequences, metadata = dqn_explorer.run(landscape)
dqn_sequences

round: 0, top: 0.27879076194752395
round: 1, top: 0.5766494494158345
round: 2, top: 0.6164433925290707
round: 3, top: 0.6164433925290707
round: 4, top: 0.6164433925290707
round: 5, top: 0.6619888241846161


sequence  model_score  true_score  round  model_cost  \
0   UCUUGGGGACUUUU          NaN    0.278791      0           0   
0   AUUACGGCGUUCAA     0.248061    0.285760      1        1000   
1   GUUUCGCGACGCUA     0.245008    0.346098      1        1000   
2   UUUAGCGCCGUCAC     0.240043    0.320321      1        1000   
3   ACUACGGCGUUCAA     0.235916    0.285760      1        1000   
..             ...          ...         ...    ...         ...   
94  UAUCGGGUCGGCUU     0.387915    0.444210      5        5000   
95  CAUCGGUCCGGCUU     0.385862    0.459392      5        5000   
96  CAUUGGGCCAGCUU     0.382194    0.399338      5        5000   
97  AAUGGGGACGGCUU     0.380385    0.471025      5        5000   
98  CCUUGCUCCGGCUA     0.380349    0.469703      5        5000   

    measurement_cost  
0                  1  
0                100  
1                100  
2                100  
3                100  
..               ...  
94               496  
95               496  
96               496  
97               496  
98               496  

[496 rows x 6 columns]

In [42]:
def cumulative_max_per_round(sequences):
    num_rounds = sequences['round'].max() + 1
    max_per_round = [sequences['true_score'][sequences['round'] == r].max()
                     for r in range(num_rounds)]

    return np.maximum.accumulate(max_per_round)

plt.figure(figsize=(10,5), dpi=300)
#plt.plot(range(11), cumulative_max_per_round(random_sequences), 'o-', label="random")
plt.plot(range(11), cumulative_max_per_round(adalead_sequences), 'o-', label="adalead")
plt.plot(range(11), cumulative_max_per_round(genetic_algo_sequences), 'o-', label="genetic algo")
#plt.plot(range(11), cumulative_max_per_round(cbas_sequences), 'o-', label="cbas")
#plt.plot(range(11), cumulative_max_per_round(ppo_sequences), 'o-', label="ppo")
plt.ylabel("Cumulative max")
plt.xlabel("Round")
plt.legend()
#plt.ylim(0, 1)
plt.show()

## 4. Evaluating an exploration algorithm <a class="anchor" id="section-four"></a>


The `flexs.evaluate` module can be used run tests aimed at different properties of the explorer. 

The tests include:
- `robustness`: shows how explorer performance changes with model quality (noisy abstract models at different signal strengths).
- `efficiency`: shows how explorer performance changes when more computational evals are allowed
- `adaptivity`: shows how the explorer is sensitive to the number of batches it is allowed to sample

Here we will test out Adalead's robustness to varying model quality.

In [80]:
def make_explorer(model, signal_strength):
    return baselines.explorers.Adalead(
            model,
            rounds=5,
            mu=1,
            starting_sequence=wt,
            sequences_batch_size=100,
            model_queries_per_batch=1000,
            alphabet=alphabet
        )

results = flexs.evaluate.robustness(landscape, make_explorer)

Evaluating for robustness with model accuracy; signal_strength: 0
round: 0, top: 0.5437468768175177, time: 0.000157s
round: 1, top: 0.7167572848082119, time: 0.729755s
round: 2, top: 0.8568133144065242, time: 1.005974s
round: 3, top: 0.8568133144065242, time: 1.307008s
round: 4, top: 0.8568133144065242, time: 1.404206s
round: 5, top: 0.8568133144065242, time: 1.359904s
Evaluating for robustness with model accuracy; signal_strength: 0.5
round: 0, top: 0.5437468768175177, time: 0.000189s
round: 1, top: 0.7304882670751143, time: 0.630952s
round: 2, top: 0.7661888000172442, time: 0.896272s
round: 3, top: 0.8842752789403289, time: 1.147306s
round: 4, top: 0.9941231370755479, time: 1.711254s
round: 5, top: 1.0215851016093527, time: 1.728895s
Evaluating for robustness with model accuracy; signal_strength: 0.75
round: 0, top: 0.5437468768175177, time: 0.000151s
round: 1, top: 0.8101279537472399, time: 0.629513s
round: 2, top: 0.9364529486991093, time: 1.006085s
round: 3, top: 1.054539480001734